In [5]:
import pandas as pd
import numpy as np
import datetime

In [105]:
Inclusions=pd.read_csv("/home/jian/Projects/Big_Lots/Q1_Products/Inclusions.csv")
Department_Selected=pd.read_excel("/home/jian/Projects/Big_Lots/Q1_Products/Department_ID_DMA.xlsx",sheetname="product_department")
DMA_Selected=pd.read_excel("/home/jian/Projects/Big_Lots/Q1_Products/Department_ID_DMA.xlsx",sheetname="selected_DMA")                                 

In [7]:
historical_data1=pd.read_csv("/home/jian/Projects/Big_Lots/Q1_Products/Data/MediaStorm Data Extract - Detail Store Sales1 (stores up to 1000).txt",dtype=str,sep="|",na_values="?")
historical_data2=pd.read_csv("/home/jian/Projects/Big_Lots/Q1_Products/Data/MediaStorm Data Extract - Detail Store Sales2 (stores 1001 to 1450).txt",dtype=str,sep="|",na_values="?")
historical_data3=pd.read_csv("/home/jian/Projects/Big_Lots/Q1_Products/Data/MediaStorm Data Extract - Detail Store Sales3 (stores 1451 to 1750).txt",dtype=str,sep="|",na_values="?")
historical_data4=pd.read_csv("/home/jian/Projects/Big_Lots/Q1_Products/Data/MediaStorm Data Extract - Detail Store Sales4 (stores 1751 to 2000).txt",dtype=str,sep="|",na_values="?")
historical_data5=pd.read_csv("/home/jian/Projects/Big_Lots/Q1_Products/Data/MediaStorm Data Extract - Detail Store Sales5 (stores 2001 to 4500).txt",dtype=str,sep="|",na_values="?")
historical_data6=pd.read_csv("/home/jian/Projects/Big_Lots/Q1_Products/Data/MediaStorm Data Extract - Detail Store Sales6 (stores 4501 to 5000).txt",dtype=str,sep="|",na_values="?")
historical_data7=pd.read_csv("/home/jian/Projects/Big_Lots/Q1_Products/Data/MediaStorm Data Extract - Detail Store Sales7 (stores 5001 to 6990).txt",dtype=str,sep="|",na_values="?")

In [8]:
def clean_date(df):
    df['week_end_dt']=df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    df=df[(df['week_end_dt']>datetime.datetime.strptime("2017-04-01","%Y-%m-%d").date()) & (df['week_end_dt']<datetime.datetime.strptime("2017-04-30","%Y-%m-%d").date())]
    df['location_id']=df['location_id'].astype(int)
    df['gross_sales_amt']=df['gross_sales_amt'].astype(float)
    df['gross_transaction_cnt']=df['gross_transaction_cnt'].astype(int)
    df['class_code_id']=df['class_code_id'].fillna(0)
    df['class_code_id']=df['class_code_id'].astype(int)
    df['class_gross_sales_amt']=df['class_gross_sales_amt'].astype(float)   
    df=df[df['location_id'].isin(Inclusions['location_id'])]
    return df

In [9]:
historical_data1=clean_date(historical_data1)
historical_data2=clean_date(historical_data2)
historical_data3=clean_date(historical_data3)
historical_data4=clean_date(historical_data4)
historical_data5=clean_date(historical_data5)
historical_data6=clean_date(historical_data6)
historical_data7=clean_date(historical_data7)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [10]:
Sales_2017=historical_data1.append(historical_data2).append(historical_data3).append(historical_data4).append(historical_data5).append(historical_data6).append(historical_data7)

In [12]:
Sales_2017.head(3)

,location_id,week_end_dt,fiscal_week_nbr,gross_sales_amt,gross_transaction_cnt,class_code_id,class_gross_sales_amt
21838,1,2017-04-08,10,111865.74,3068,11001,1135.92
21839,1,2017-04-08,10,111865.74,3068,11002,845.72
21840,1,2017-04-08,10,111865.74,3068,11003,4034.92


In [13]:
store_DMA=pd.read_excel("/home/jian/BigLots/static_files/all_store_DMA_2017.xlsx")
store_DMA=store_DMA[['location_id','DMA']]

In [14]:
Product_Taxonomy=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Product Taxonomy.txt",sep="|")
Product_Taxonomy=Product_Taxonomy[['class_code_id','department_id']]
Product_Taxonomy=Product_Taxonomy.drop_duplicates()
Product_Department_Name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",sep="|")
Product_Taxonomy=pd.merge(Product_Taxonomy,Product_Department_Name,on="department_id",how="left")

In [15]:
Sales_2017=pd.merge(Sales_2017,Product_Taxonomy,on="class_code_id",how="left")

In [72]:
Sales_2017_department=Sales_2017.groupby(['location_id','week_end_dt','department_id','department_desc'])['class_gross_sales_amt'].sum().to_frame()
Sales_2017_department.reset_index(inplace=True)

In [73]:
Sales_2017_department_DMA=pd.merge(Sales_2017_department,store_DMA,on="location_id",how="left")

In [74]:
Sales_2017_department_DMA_April=Sales_2017_department_DMA.groupby(['DMA','department_desc'])['class_gross_sales_amt'].sum().to_frame().reset_index()

In [75]:
Sales_2017_department_DMA_April=Sales_2017_department_DMA_April.sort_values(['DMA','class_gross_sales_amt'],ascending=[True,False])

In [76]:
Sales_2017_all_DMA_April=Sales_2017_department_DMA_April.groupby(['DMA'])['class_gross_sales_amt'].sum().to_frame().reset_index()
Sales_2017_all_DMA_April.columns=['DMA','Total_Sales']

In [77]:
Sales_2017_department_DMA_April=pd.merge(Sales_2017_department_DMA_April,Sales_2017_all_DMA_April,on="DMA",how="left")

In [78]:
Sales_2017_department_DMA_April['Product_Sales_Pctg']=Sales_2017_department_DMA_April['class_gross_sales_amt']/Sales_2017_department_DMA_April['Total_Sales']


In [79]:
Df_DMA_Top8_Products=pd.DataFrame(columns=['DMA','Top1_Product','Top1_Share','Top2_Product','Top2_Share','Top3_Product','Top3_Share',
                                 'Top4_Product','Top4_Share','Top5_Product','Top5_Share','Top6_Product','Top6_Share',
                                 'Top7_Product','Top7_Share','Top8_Product','Top8_Share'])
for dma in Sales_2017_department_DMA_April['DMA'].unique():
    df=Sales_2017_department_DMA_April[Sales_2017_department_DMA_April['DMA']==dma]
    df.reset_index(inplace=True)
    del df['index']
    df_app=pd.DataFrame(columns=['DMA','Top1_Product','Top1_Share','Top2_Product','Top2_Share','Top3_Product','Top3_Share',
                                 'Top4_Product','Top4_Share','Top5_Product','Top5_Share','Top6_Product','Top6_Share',
                                 'Top7_Product','Top7_Share','Top8_Product','Top8_Share'],index=[0])
    df_app['DMA'][0]=dma
    df_app['Top1_Product'][0]=df['department_desc'][0]
    df_app['Top1_Share'][0]=df['Product_Sales_Pctg'][0]
    df_app['Top2_Product'][0]=df['department_desc'][1]
    df_app['Top2_Share'][0]=df['Product_Sales_Pctg'][1]
    df_app['Top3_Product'][0]=df['department_desc'][2]
    df_app['Top3_Share'][0]=df['Product_Sales_Pctg'][2]
    df_app['Top4_Product'][0]=df['department_desc'][3]
    df_app['Top4_Share'][0]=df['Product_Sales_Pctg'][3]
    df_app['Top5_Product'][0]=df['department_desc'][4]
    df_app['Top5_Share'][0]=df['Product_Sales_Pctg'][4]
    df_app['Top6_Product'][0]=df['department_desc'][5]
    df_app['Top6_Share'][0]=df['Product_Sales_Pctg'][5]
    df_app['Top7_Product'][0]=df['department_desc'][6]
    df_app['Top7_Share'][0]=df['Product_Sales_Pctg'][6]
    df_app['Top8_Product'][0]=df['department_desc'][7]
    df_app['Top8_Share'][0]=df['Product_Sales_Pctg'][7]
    Df_DMA_Top8_Products=Df_DMA_Top8_Products.append(df_app)
    
Df_DMA_Top8_Products.reset_index(inplace=True)
del Df_DMA_Top8_Products['index']

In [106]:
Df_DMA_Top8_Products_Seleceted_13DMAs=Df_DMA_Top8_Products[Df_DMA_Top8_Products['DMA'].isin(DMA_Selected['DMA'])]

In [107]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Q1_Products/BL_April_2017_store_DMA_JL_20180322.xlsx",engine='xlsxwriter')

In [108]:
Df_DMA_Top8_Products_Seleceted_13DMAs.to_excel(writer,"April_top8_products_13_DMAs",index=False)
Df_DMA_Top8_Products.to_excel(writer,"April_top8_products_all_DMAs",index=False)
Sales_2017_department_DMA_April.to_excel(writer,"data_by_DMA_product",index=False)
Sales_2017_department.to_excel(writer,"data_by_week_store_product",index=False)

In [109]:
writer.save()